# Porównanie klasyfikatora konwolucyjnego i rekurencyjnego dla NLP

W tym notebooku porównamy działanie (architekturę, złożoność, efektywność uczenia) dwóch klasyfikatorów sentymentu recenzji

#### Załadujmy zależności

In [ ]:
import tensorflow
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding
from tensorflow.keras.layers import SpatialDropout1D, Conv1D, GlobalMaxPooling1D, SimpleRNN
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score, roc_curve, auc, RocCurveDisplay
import matplotlib.pyplot as plt

#### Ustawmy hiperparametry dla klasyfikatora konwolucyjnego

In [ ]:
# lokalizacja wyjściowa dla wag:
conv_output_dir = 'model_output/conv'

# parametry treningu:
conv_epochs = 4
conv_batch_size = 128

# osadzenie przestrzeni wektorowej: 
conv_n_dim = 64
conv_n_unique_words = 5000 
conv_max_review_length = 400
conv_pad_type = trunc_type = 'pre'

#parametr dropoutu
conv_drop_embed = 0.2

# parametry architektury konwolucyjnej:
n_conv = 256 # filtry/kernele
k_conv = 3 # długość kernela

# parametry warstwy gęstej: 
n_dense = 256
dropout = 0.2

#### Ustawmy hiperparametry dla klasyfikatora rekurencyjnego

In [ ]:
# lokalizacja wyjściowa dla wag:
rnn_output_dir = 'model_output/rnn'

# parametry treningu:
rnn_epochs = 16
rnn_batch_size = 128

# osadzenie przestrzeni wektorowej: 
rnn_n_dim = 64 
rnn_n_unique_words = 10000 
rnn_max_review_length = 100 #zmniejszone z powodu szybciej zanikającego gradientu w jednostkach rekurencyjnych
rnn_pad_type = trunc_type = 'pre'
rnn_drop_embed = 0.2 

# parametry architektury rekurencyjnej:
n_rnn = 256 
drop_rnn = 0.2

#### Załaduj dane treningowe i walidacyjne z parametrem liczby uwzględnianych słów (odrębnie dla obu klasyfikatorów)

In [ ]:
(x_train_rnn, y_train_rnn), (x_val_rnn, y_val_rnn) = imdb.load_data(num_words=conv_n_unique_words)
(x_train_conv, y_train_conv), (x_val_conv, y_val_conv) = imdb.load_data(num_words=rnn_n_unique_words)

#### Przetwórz wszystkie załadowane dane poprzez ujednolicenie wielkości danych wejściowych

In [ ]:
#dla rnn
x_train_rnn = pad_sequences(x_train_rnn, maxlen=rnn_max_review_length, padding=rnn_pad_type, truncating=trunc_type)
x_val_rnn = pad_sequences(x_val_rnn, maxlen=rnn_max_review_length, padding=rnn_pad_type, truncating=trunc_type)

In [ ]:
# dla conv
x_train_conv = pad_sequences(x_train_conv, maxlen=conv_max_review_length, padding=conv_pad_type, truncating=trunc_type)
x_val_conv = pad_sequences(x_val_conv, maxlen=conv_max_review_length, padding=conv_pad_type, truncating=trunc_type)

#### Zaprojektuj obie architektury sieci, wybierając odpowiednie zaimportowane obiekty i zadeklarowane wyżej hiperparametry

#### ConvNN
- odpowiednio osadź przestrzeń wektorową (embedding), 
- dodaj przestrzenny dropout z odpowiednim parametrem i wybraną funkcją aktywacji
- utwórz warstwę konwolucyjną jednowymiarową z odpowiednimi hiperparametrami
- dodaj warstwę redukującą - globalny jednowymiarowy max-pooling
- uzupełnij sieć o warstwę gęstą z wybraną funkcją aktywacji i dropoutem
- zakończ warstwą klasyfikującą

In [ ]:
model_conv = Sequential([
    Embedding(input_dim=conv_n_unique_words, output_dim=conv_n_dim, input_length=conv_max_review_length),
    SpatialDropout1D(conv_drop_embed),
    Conv1D(filters=n_conv, kernel_size=k_conv, padding='same', activation='relu'),
    GlobalMaxPooling1D(),
    Dense(n_dense, activation='relu'),
    Dropout(dropout),
    Dense(1, activation='sigmoid')
])

In [ ]:
model_conv.summary() 

#### RNN
- odpowiednio osadź przestrzeń wektorową (embedding), 
- dodaj przestrzenny dropout z odpowiednim parametrem i wybraną funkcją aktywacji
- utwórz prostą warstwę rekurencyjną z odpowiednimi hiperparametrami
- zakończ warstwą klasyfikującą

In [ ]:
model_rnn = Sequential([
    Embedding(input_dim=rnn_n_unique_words, output_dim=rnn_n_dim, input_length=rnn_max_review_length),
    SpatialDropout1D(rnn_drop_embed),
    SimpleRNN(n_rnn, return_sequences=True, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model_rnn.summary()

#### Skompiluj oba modele z odpowiednimi parametrami

In [ ]:
# compile the model:
model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_rnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Stwórz obiekty do rejestrowania wag dla każdego z modeli

In [ ]:
# dla rnn
checkpoint_rnn = ModelCheckpoint(os.path.join(rnn_output_dir, 'rnn_weights.h5'), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
if not os.path.exists(rnn_output_dir):
    os.makedirs(rnn_output_dir)

In [ ]:
# dla conv
checkpoint_conv = ModelCheckpoint(os.path.join(conv_output_dir, 'conv_weights.h5'), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
if not os.path.exists(conv_output_dir):
    os.makedirs(conv_output_dir)

#### Naucz oba modele z użyciem zbiorów walidacyjnych, z odpowiednimi zadeklarowanymi wyżej parametrami

In [ ]:
# rnn
history_rnn = model_rnn.fit(x_train_rnn, y_train_rnn, epochs=rnn_epochs, batch_size=rnn_batch_size, validation_data=(x_val_rnn, y_val_rnn), callbacks=[checkpoint_rnn])

In [ ]:
# conv
history_conv = model_conv.fit(x_train_conv, y_train_conv, epochs=conv_epochs, batch_size=conv_batch_size, validation_data=(x_val_conv, y_val_conv), callbacks=[checkpoint_conv])

#### Wykonaj inferencję (ewaluację) obu modeli uprzednio ładując wagi z ich najlepszych epok

In [ ]:
model_conv.load_weights(os.path.join(conv_output_dir, 'conv_weights.04-0.00.h5'))
model_rnn.load_weights(os.path.join(rnn_output_dir, 'rnn_weights.04-0.00.h5'))

In [ ]:
# predict
y_pred_conv = model_conv.predict(x_val_conv)
y_pred_rnn = model_rnn.predict(x_val_rnn)


#### Zestaw na jednym wykresie histogramy dla danych walidacyjnych obu modeli oraz, na kolejnym wykresie, krzywe ROC obu klasyfikatorów,  a następnie - patrząc na oba te wykresy oraz statystyki uczenia, zapisz trzy sensowne wnioski co do wyników

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(y_pred_conv, bins=20, label='conv')
ax.hist(y_pred_rnn, bins=20, label='rnn')
ax.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(roc_curve(y_val_conv, y_pred_conv), label='conv')
ax.plot(roc_curve(y_val_rnn, y_pred_rnn), label='rnn')
ax.legend()
plt.show()


**tu wnioski**